In [1]:
import os
os.environ['TRANSFORMERS_CACHE'] = '/media/pc3/4172eaa0-8170-450a-8016-28e8ffd4f9ed/HTR_FIRAT/transformers/'

from transformers import MBartForConditionalGeneration, MBart50TokenizerFast,\
                        Seq2SeqTrainingArguments, Seq2SeqTrainer, DataCollatorForSeq2Seq
from datasets import load_dataset, load_metric

## Model

In [2]:
model = MBartForConditionalGeneration.from_pretrained("facebook/mbart-large-50")

In [3]:
tokenizer = MBart50TokenizerFast.from_pretrained("facebook/mbart-large-50", src_lang="en_XX", tgt_lang="tr_TR")

In [4]:
sample_en_sentence = 'hello, how is it going?'
tr_correspondence  = 'merhaba, nasıl gidiyor?'

inputs = tokenizer(sample_en_sentence, return_tensors="pt")
with tokenizer.as_target_tokenizer():
    labels = tokenizer(tr_correspondence, return_tensors="pt")

translated_tokens = model.generate(**inputs, decoder_start_token_id=tokenizer.lang_code_to_id["tr_TR"])

In [5]:
metric = load_metric("sacrebleu")

## Data

In [6]:
prefix = ""
max_input_length = 128
max_target_length = 128
source_lang = "en"
target_lang = "tr"

def preprocess_function(examples):
    inputs = [prefix + ex[source_lang] for ex in examples["translation"]]
    targets = [ex[target_lang] for ex in examples["translation"]]
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True)

    # Setup the tokenizer for targets
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(targets, max_length=max_target_length, truncation=True)
    model_inputs["labels"] = labels["input_ids"]

    return model_inputs

In [7]:
dataset = load_dataset("wmt16", "tr-en")

Reusing dataset wmt16 (/home/pc3/.cache/huggingface/datasets/wmt16/tr-en/1.0.0/af3c5d746b307726d0de73ebe7f10545361b9cb6f75c83a1734c000e48b6264f)


  0%|          | 0/3 [00:00<?, ?it/s]

In [8]:
preprocessed_dataset = dataset.map(preprocess_function, batched=True)

Loading cached processed dataset at /home/pc3/.cache/huggingface/datasets/wmt16/tr-en/1.0.0/af3c5d746b307726d0de73ebe7f10545361b9cb6f75c83a1734c000e48b6264f/cache-f1a8661a4757d607.arrow


  0%|          | 0/2 [00:00<?, ?ba/s]

Loading cached processed dataset at /home/pc3/.cache/huggingface/datasets/wmt16/tr-en/1.0.0/af3c5d746b307726d0de73ebe7f10545361b9cb6f75c83a1734c000e48b6264f/cache-a5e7fecb22426b35.arrow


In [9]:
checkpoint_path = '/media/pc3/4172eaa0-8170-450a-8016-28e8ffd4f9ed/HTR_FIRAT/transformer-chekpoint'
batch_size = 1

args = Seq2SeqTrainingArguments(
    output_dir = checkpoint_path,
    evaluation_strategy = 'epoch',
    learning_rate = 1e-4,
    per_device_train_batch_size = batch_size,
    per_device_eval_batch_size = batch_size,
    weight_decay = 0.01,
    save_total_limit = 3,
    num_train_epochs = 1,
    predict_with_generate = True
)

In [10]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [11]:
def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [[label.strip()] for label in labels]
    return preds, labels

def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]

    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Some simple post-processing
    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)
    result = metric.compute(predictions=decoded_preds, references=decoded_labels)
    result = {"bleu": result["score"]}
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]
    result["gen_len"] = np.mean(prediction_lens)
    result = {k: round(v, 4) for k, v in result.items()}

    return result

In [12]:
trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=preprocessed_dataset["train"],
    eval_dataset=preprocessed_dataset["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [13]:
trainer.train()

The following columns in the training set  don't have a corresponding argument in `MBartForConditionalGeneration.forward` and have been ignored: translation.
***** Running training *****
  Num examples = 205756
  Num Epochs = 1
  Instantaneous batch size per device = 1
  Total train batch size (w. parallel, distributed & accumulation) = 1
  Gradient Accumulation steps = 1
  Total optimization steps = 205756


RuntimeError: CUDA out of memory. Tried to allocate 978.00 MiB (GPU 0; 9.78 GiB total capacity; 6.46 GiB already allocated; 937.69 MiB free; 6.98 GiB reserved in total by PyTorch)